## Utility functions

In [ ]:
def between(x1, x2):
    return int((x2 - x1) * 0.5) + x1


def make_dir_if_absent(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

## Generate sine tones

In [ ]:
make_dir_if_absent('data/test_sines')
! csound gen_test_tones.csd --omacro:TEST_TYPE=1 -f --output=data/test_sines/sweep_down.wav
! csound gen_test_tones.csd --omacro:TEST_TYPE=2 -f --output=data/test_sines/sweep_up.wav
! csound gen_test_tones.csd --omacro:TEST_TYPE=3 -f --output=data/test_sines/sweep_both.wav

## Generate noises
### Create a list with modulation parameters

In [ ]:
def filter_freq_pairs(tupl):
    (f1l, f1h), (f2l, f2h) = tupl
    c1 = between(f1l, f1h)
    c2 = between(f2l, f2h)
    return abs(c1 - c2) < 5000


def gen_freq_list():
    return [(int(f1), int(f2))
            for (f1, f2)
            in combinations(np.linspace(0, 20000, 100), 2)
            if f2 > f1]


freqs = gen_freq_list()
pair_freqs = filter(filter_freq_pairs,
                    combinations(freqs, 2))[::200]
total_combs = len(pair_freqs)

### Sound generation routines

In [ ]:
noises_dir = 'data/test_noises/'
cmd_template = 'csound gen_noises.csd --omacro:cf1={} --omacro:bw1={} ' \
               '--omacro:cf2={} --omacro:bw2={} -f --output={}'

    
def cmp_as_string(x, y):
    if x > y:
        return 'down'
    elif x < y:
        return 'up'
    else:
        return 'still'


def render_file(low1, high1, low2, high2):
    bw1 = high1 - low1
    cf1 = low1 + int(bw1 * 0.5)
    bw2 = high2 - low2
    cf2 = low2 + int(bw2 * 0.5)
    wav_name = '{}-{}--{}-{}--{}.wav'.format(cf1, bw1, cf2, bw2,
                                             cmp_as_string(cf1, cf2))
    if os.path.isfile(noises_dir + wav_name):
        return Popen('false')
    cmd = cmd_template.format(cf1, bw1, cf2, bw2, noises_dir + wav_name)
    return Popen(cmd.split(' '))


def run_parallel(pair_freq_list, num_threads=4):
    procs = set()
    count = 0
    for (low1, high1), (low2, high2) in pair_freq_list:
        count += 1
        sys.stdout.write('{} / {}\r'.format(count, total_combs))
        procs.add(render_file(low1, high1, low2, high2))
        if len(procs) > num_threads:
            os.wait()  # won't work on Windows
            procs -= {p for p in procs if p.poll() is not None}

### Run generation

In [ ]:
make_dir_if_absent(noises_dir)
run_parallel(pair_freqs, num_threads=8)